In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cse493g1/assignments/assignment2/'
FOLDERNAME = 'cse493g1/cse493g1project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

from model_trainer import Trainer
from Model import GraphCaptioningModel
from model_utils import decode_captions, create_minibatch, encode_captions

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import ast
import re

class GraphImageDataset(Dataset):
    def __init__(self, csv_files, transform=None):
        self.data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      x, y = self.data.iloc[idx]
      x_out = str(x)
      y_out = str(y)
      return x_out, y_out

dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [4]:
csv_files1 = ['/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_kk0.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_cr0.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_gv0.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_sp0.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_kk0_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_cr0_medium.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_gv0_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_sp0_medium.csv']

csv_files2 = ['/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_kk0.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_cr0.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_gv0.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_sp0.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_kk0_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_cr0_medium.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_gv0_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_sp0_medium.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_kk1.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_cr1.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_gv1.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_small/data_sp1.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_kk1_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_cr1_medium.csv',
              '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_gv1_medium.csv', '/content/drive/My Drive/cse493g1/cse493g1project/datasets/datasets_medium/data_sp1_medium.csv']
dataset_default = GraphImageDataset(csv_files=csv_files1)
dataset_mixed = GraphImageDataset(csv_files=csv_files2)

In [5]:
print(dataset_mixed.__getitem__(2500))

('/datasets_small/graph_images_sp_color/sp1_gi20.png', '{16: [(16, 17), (16, 19), (16, 48), (16, 84), (16, 98)], 17: [(16, 17), (17, 46), (17, 48), (17, 59), (17, 80), (17, 84), (17, 98)], 19: [(16, 19), (19, 38), (19, 43), (19, 46), (19, 48), (19, 59), (19, 80), (19, 84), (19, 98)], 38: [(19, 38), (38, 38), (38, 43), (38, 46), (38, 48), (38, 59), (38, 84), (38, 98)], 43: [(19, 43), (38, 43), (43, 46), (43, 59), (43, 80), (43, 98)], 46: [(17, 46), (19, 46), (38, 46), (43, 46), (46, 48), (46, 59), (46, 80), (46, 84), (46, 98)], 48: [(16, 48), (17, 48), (19, 48), (38, 48), (46, 48), (48, 59), (48, 80), (48, 98)], 59: [(17, 59), (19, 59), (38, 59), (43, 59), (46, 59), (48, 59), (59, 80)], 80: [(17, 80), (19, 80), (43, 80), (46, 80), (48, 80), (59, 80)], 84: [(16, 84), (17, 84), (19, 84), (38, 84), (46, 84)], 98: [(16, 98), (17, 98), (19, 98), (38, 98), (43, 98), (46, 98), (48, 98), (98, 98)]}')


In [ ]:
import PIL
import torchvision.transforms.functional as F
from torchvision.utils import make_grid
from torchvision.io import read_image
import torchvision.transforms as transform
from pathlib import Path

raw_data_nc = {}
nc_len = dataset_default.__len__()
raw_data_clr = {}
clr_len = dataset_mixed.__len__()

"""
graph_list = []
caption_list = []
for i in range(nc_len):
  graph_path, caption = dataset_default.__getitem__(i)
  graph = F.pil_to_tensor(PIL.Image.open('/content/drive/My Drive/cse493g1/cse493g1project/datasets' + graph_path).convert('RGB'))
  graph_list.append(np.array([graph.numpy()]).reshape(graph.shape))
  caption_list.append(caption)
raw_data_nc['features'] = np.array(graph_list)
raw_data_nc['captions'] = np.array(caption_list)
"""

graph_list = []
caption_list = []
for i in np.random.choice(4640, 3000):
  graph_path, caption = dataset_mixed.__getitem__(i)
  graph = F.pil_to_tensor(PIL.Image.open('/content/drive/My Drive/cse493g1/cse493g1project/datasets' + graph_path).convert('RGB'))
  graph_list.append(np.array([graph.numpy()]).reshape(graph.shape))
  caption_list.append(caption)
raw_data_clr['features'] = np.array(graph_list)
raw_data_clr['captions'] = np.array(caption_list)

"""
graph1 = F.pil_to_tensor(PIL.Image.open('/content/drive/My Drive/cse493g1/cse493g1project/gi0.png').convert('RGB'))
#graph2 = F.pil_to_tensor(PIL.Image.open('gi1.png').convert('RGB'))
#graph3 = F.pil_to_tensor(PIL.Image.open('gi3.png').convert('RGB'))
raw_data['features'] = np.array([graph1.numpy()]).reshape(graph1.shape)
raw_data['features'] = raw_data['features'][:,:512,:512]
raw_data['captions'] = np.array(["{74: [(85, 74), (74, 65), (32, 74), (79, 74)], 32: [(27, 32), (32, 85), (32, 78), (32, 74), (32, 78)], 45: [(79, 45), (45, 45), (45, 33)], 64: [(79, 64), (30, 64), (43, 64)], 78: [(65, 78), (85, 78), (32, 78), (32, 78), (65, 78), (78, 33)], 85: [(85, 74), (85, 79), (32, 85), (85, 78), (85, 27), (33, 85)], 43: [(43, 43), (43, 64), (65, 43)], 63: [], 5: [(79, 5), (79, 5), (36, 5)], 33: [(33, 79), (45, 33), (78, 33), (38, 33), (33, 72), (33, 85)], 27: [(27, 32), (27, 72), (27, 79), (85, 27)], 79: [(79, 45), (79, 64), (85, 79), (33, 79), (79, 5), (72, 79), (72, 79), (79, 74), (79, 36), (79, 5), (27, 79)], 38: [(38, 65), (36, 38), (38, 33)], 36: [(36, 38), (79, 36), (72, 36), (36, 5)], 30: [(30, 64)], 72: [(27, 72), (72, 79), (72, 79), (72, 36), (33, 72)], 65: [(38, 65), (74, 65), (65, 78), (65, 65), (65, 78), (65, 43)]}"])
"""

In [ ]:
"""
print(raw_data_nc['features'].shape)
print(raw_data_nc['features'][0])
print(raw_data_nc['features'][0].shape)
print(raw_data_nc['captions'].shape)
print(raw_data_nc['captions'][0])
print(raw_data_nc['captions'][0].shape)
"""
print("MIXED COLOR DATA")

print(raw_data_clr['features'].shape)
print(raw_data_clr['features'][0])
print(raw_data_clr['features'][0].shape)
print(raw_data_clr['captions'].shape)
print(raw_data_clr['captions'][0])
print(raw_data_clr['captions'][0].shape)

In [ ]:
from Model import GraphCaptioningModel
data = {}

data['idx_to_word'] = ['<NULL>', '<START>', '<END>']
for i in range(100):
  data['idx_to_word'].append(str(i))
punc = ['{', '}', '[', ']', '(', ')', ':', ',', ' ']
for p in punc:
  data['idx_to_word'].append(p)

data['word_to_idx'] = {}
for i in range(len(data['idx_to_word'])):
  data['word_to_idx'][data['idx_to_word'][i]] = i

data['train_captions'] = torch.tensor(encode_captions(raw_data['captions'], data['word_to_idx'])).type(dtype)
data['train_features'] = torch.tensor(np.array([raw_data['features']])).type(dtype)
print(data['train_features'].shape)
print(data['train_captions'].shape)

transformer = GraphCaptioningModel(
          word_to_idx=data['word_to_idx'],
          wordvec_dim=256,
          max_length=2000
        ).type(dtype)


transformer_solver = Trainer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=10,
           batch_size=1,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

NameError: ignored

In [ ]:
from Model import GraphCaptioningModel
torch.manual_seed(231)
np.random.seed(231)

import gc

gc.collect()

data_clr = {}

data_clr['idx_to_word'] = ['<NULL>', '<START>', '<END>']
for i in range(100):
  data_clr['idx_to_word'].append(str(i))
punc = ['{', '}', '[', ']', '(', ')', ':', ',', ' ']
for p in punc:
  data_clr['idx_to_word'].append(p)

data_clr['word_to_idx'] = {}
for i in range(len(data_clr['idx_to_word'])):
  data_clr['word_to_idx'][data_clr['idx_to_word'][i]] = i

tenth = len(raw_data_clr)//10

encoded_captions = encode_captions(raw_data_clr['captions'], data_clr['word_to_idx'])

data_clr['train_captions'] = torch.tensor(encoded_captions[:tenth*8]).type(dtype)
data_clr['train_features'] = torch.tensor(raw_data_clr['features'][:tenth*8]).type(dtype)

data_clr['val_captions'] = torch.tensor(encoded_captions[tenth*8:tenth*9]).type(dtype)
data_clr['val_features'] = torch.tensor(raw_data_clr['features'][tenth*8:tenth*9]).type(dtype)

data_clr['test_captions'] = torch.tensor(encoded_captions[tenth*9:]).type(dtype)
data_clr['test_features'] = torch.tensor(raw_data_clr['features'][tenth*9:]).type(dtype)

gc.collect()

graph_model_clr = GraphCaptioningModel(
          word_to_idx=data['word_to_idx'],
          wordvec_dim=256,
          max_length=1600
        ).type(dtype)

gc.collect()

model_solver_clr = Trainer(graph_model_clr, data_clr, idx_to_word=data['idx_to_word'],
           num_epochs=10,
           batch_size=10,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

model_solver_clr.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

In [ ]:
for split in ['train', 'val']:
    minibatch = create_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()